In [23]:
import pandas as pd
import utils

In [24]:
def time_point(modality, time_point):
    """
    Convert time point in hr to long or short time description
    Parameters:
    -----------
    modality: str
        perturbation modality
    time_point: int
        time point in hr
    Returns:
    -------
    str of time description
    """
    if modality == "compound":
        if time_point == "24":
            time = "short"
        else:
            time = "long"
    elif modality == "orf":
        if time_point == "48":
            time = "short"
        else:
            time = "long"
    else:
        if time_point == "96":
            time = "short"
        else:
            time = "long"

    return time

In [25]:
# Matric results were pre-calcuated and saved in a parquet file. Those are read here.

cols = [
    "Metadata_Perturbation",
    "Metadata_Time",
    "Metadata_Cell_type",
    "Metadata_broad_sample",
    "sim_retrieval_average_precision_ref_i",
    "sim_retrieval_average_precision_ref_i_adjusted",
]

results = (
    pd.read_parquet('../compare_parquet/metrics_level_1_0_ref_null_adjusted.parquet')[cols]
)

results["Metadata_Time"] = results["Metadata_Time"].astype(str)
results["Timepoint"] = results.apply(lambda x: time_point(x['Metadata_Perturbation'], x['Metadata_Time']), axis=1)
results["Cell_Time"] = results[["Metadata_Cell_type", "Timepoint"]].agg(
    "_".join, axis=1
)


In [28]:
results = (
    results.groupby(["Metadata_Perturbation", "Cell_Time", "Metadata_broad_sample"])[
        [
            "sim_retrieval_average_precision_ref_i",
            "sim_retrieval_average_precision_ref_i_adjusted",
        ]
    ]
    .mean()
    .reset_index()
    .rename(
        columns={
            "sim_retrieval_average_precision_ref_i": "mAP",
            "sim_retrieval_average_precision_ref_i_adjusted": "adjusted_mAP",
        }
    )
)

results.dropna(inplace=True)

results.to_csv("output/matric-ap.csv", index=False)